In [24]:
import csv
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools as it
from loguru import logger
import time
# this is where any files or directories/path variables go:
csv_filename = './meetings.csv'


## SETUP A LOGURU LOGGER:
<hr>

In [25]:
log_file_path = './data/meeting_scraper.log'
rotation="100 MB"

In [26]:
logger.add(log_file_path, rotation=rotation, level="INFO", format="[<green>{time: MMM D YYYY HH:mm:ss:SSSS}</>] | <level>{message}</>", backtrace=True, diagnose=True)

2

## BUILDING A BASIC SCRAPER:
<hr>

In [27]:

def create_list_from_column_data(csv_filename, column):
    """
    :description: Create a list from a CSV column's data. To do this I will use the with open()
    context manager and open the csv file that was passed to this function as an argument.
    ----------------------------------------------------------------------------------------------------
    :param csv_filename:-> This is the name of the CSV file containing the column data I want to extract
    to create a list.
    :param column:-> This is the column name or number to extract data from in the CSV file.
    ----------------------------------------------------------------------------------------------------
    :return:-> A list of strings/integers/float/datetypes from a CSV file.
    """
    # now create an empty list to append data to:
    data_list = []
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        _column = next(csv_reader)
        columnIndex = _column.index(column)
        # loop through CSV file and append to address_list
        for line in csv_reader:
            all_data = line[columnIndex]
            data_list.append(all_data)
    return data_list


In [28]:
def fetch_soup_data(link):
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")
    logger.info("Retrieved soup_data for link: {}", link)
    return soup
    

In [29]:
# CREATE A FUNCTION THAT WILL EXTRACT ADDRESS DATA FROM EACH LINK IN 
# LINK LIST:
def get_address_data(soup):
    
    try:
        address_tag = soup.address
        address = address_tag.contents[1]
        
        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]
        
        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]
        
        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        logger.info("[+] Address data retrieved")
        return {'name': name, 'address': address, 'city': city, 'state': state}
        
    except IndexError as ie:
        logger.error("[-] UnboundError occured {} ", ie)
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundLocalError occured: {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)

In [30]:
# CREATE A FUNCTION THAT WILL EXTRACT ALL THE TABLE DATA FROM EACH LINK
# IN THE LINK LIST. THE TABLE DATA WILL THEN NEED TO BE PARSED AND 
# CLEANED IF THERE ARE MULTIPLE ITEMS:
def get_table_data(soup):
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

    
        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        logger.info("[+] Meeting Details Retrieved")
        # now return data
        return {'day': day, 'time': time, 'info': info}
    
    except AttributeError as ae:
        logger.error("info_table.find_all('tr') failed: {}", ae)
        return {'day': 'day', 'time': 'time', 'info': 'info'}

In [31]:
# CREATE A FUNCTION THAT WILL PARSE THE ROW DATA AND STORE IT 
# IN A DICTIONARY. THAT DICTIONARY CAN THEN BE INSERTED INTO 
# A LIST OF DICTIONARIES CALLED ROW_LIST BUT TO DO THIS THE PARSER
# HAS TO JOIN LIST ITEMS INTO ONE LONG STRING SO EACH ROW HAS THE 
# SAME NUMBER OF COLUMNS:
# THIS WAS INSERTED INTO meeting_data_scraper

# VERSION 1
def parse_dicts(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {'name':[], 'address':[], 'city':[], 'state':[], 'zip_code':[], 'day':[], 'time':[], 'info':[]}
    try:

        try:
            row['name'].append(item0['name'])
            row['address'].append(item0['address'])
            row['city'].append(item0['city'])
            row['state'].append(item0['state'])
            row['zip_code'].append('00000')
        except Exception as e:
            logger.error("[-] Row Data Raised Exception: {}", e)
            print(e)
            row['name'].append('name')
            row['address'].append('address')
            row['city'].append('city')
            row['state'].append('state')
            row['zip_code'].append('zip_code')

        # now add item1 to the row data
        row['day'].append(' | '.join(item1['day']))
        row['time'].append(' | '.join(item1['time']))
        row['info'].append(' | '.join(item1['info']))

        logger.info("[+] Row Data Parsed")
        return row
    except Exception as e:
        logger.error("Exception Raised: {}", e)
    


In [32]:
# THIS IS THE 'MAIN LOGICAL FUNCTION' THIS FUNCTION WILL COMBINE THE 
# get_address_data, get_table_data, and meeting_row_parser FUNCTIONS.
# THAT WAY I CAN EXECUTE ALL OF THE FUNCTIONS IN ONE CALL.
def meeting_data_scraper(link):
    
    # Get Soup Data
    soup = fetch_soup_data(link)
    # Create two dicts with the following keys
    address_dict = get_address_data(soup)
    details_dict = get_table_data(soup)

    d = [address_dict, details_dict]
    row_data = row_parser1(d[0], d[1])

    logger.info("[+] Main Meeting Dart Scraper Worked")
    return row_data


In [33]:
def flatten(list_of_lists):
    "Flatten one level of nesting"
    return it.chain.from_iterable(list_of_lists)

<hr>

In [34]:
def create_list_from_column_data(csv_filename, column):
    """
    :description: Create a list from a CSV column's data. To do this I will use the with open()
    context manager and open the csv file that was passed to this function as an argument.
    ----------------------------------------------------------------------------------------------------
    :param csv_filename:-> This is the name of the CSV file containing the column data I want to extract
    to create a list.
    :param column:-> This is the column name or number to extract data from in the CSV file.
    ----------------------------------------------------------------------------------------------------
    :return:-> A list of strings/integers/float/datetypes from a CSV file.
    """
    # now create an empty list to append data to:
    data_list = []
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        _column = next(csv_reader)
        columnIndex = _column.index(column)
        # loop through CSV file and append to address_list
        for line in csv_reader:
            all_data = line[columnIndex]
            data_list.append(all_data)
    return data_list


def fetch_soup_data(link):
    """
    Basic function that takes a link from link_list and accepts it as an argument, 
    then uses the requests.get method to return an html page. Then uses BeautifulSoup 
    to parse the pages' text using lxml
    """
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")
    logger.info("got soup data for link: {}", link)
    return soup

In [35]:
# CREATE A FUNCTION THAT WILL EXTRACT ADDRESS DATA FROM EACH LINK IN 
# LINK LIST:
def get_address_data(soup):
    """
    :description: Extracts the address data from a soup object that gets past to it in a 
    for loop. 

    :param soup: This is a BeautifulSoup object that has been pulled from a soup_data list that was 
    created using the fetch_soup_data function and stored into a soup_data list.

    :returns: A dictionary containing the address data components: {address, name, city, state}

    ::: I need to figure out how to add zip_code to this :::
    """
    try:
        address_tag = soup.address
        address = address_tag.contents[1]
        
        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]
        
        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]
        
        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        logger.info("[+] Address data retrieved")
        return {'name': name, 'address': address, 'city': city, 'state': state}
        
    except IndexError as ie:
        logger.error("[-] UnboundError occured {} ", ie)
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundLocalError occured: {} ", ule)
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            logger.error("[-] UnboundError occured {} ", ule)


# CREATE A FUNCTION THAT WILL EXTRACT ALL THE TABLE DATA FROM EACH LINK
# IN THE LINK LIST. THE TABLE DATA WILL THEN NEED TO BE PARSED AND 
# CLEANED IF THERE ARE MULTIPLE ITEMS:
def get_table_data(soup):
    """
    :description: Extracts the meeting details data from a soup object that gets past to it in a 
    for loop. The data is extracted from an html table. Each row that returns a list is joined together to make 
    one string so it can be saved to a csv file.

    :param soup: This is a BeautifulSoup object that has been pulled from a soup_data list that was 
    created using the fetch_soup_data function and stored into a soup_data list.

    :returns: a dictionary with the days the meeting runs, the time, and info about the meeting: 
    {day, time, info}
    """
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

    
        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        logger.info("[+] Meeting Details Retrieved")
        # now return data
        return {'day': day, 'time': time, 'info': info}
    
    except AttributeError as ae:
        logger.error("info_table.find_all('tr') failed: {}", ae)
        return {'day': 'day', 'time': 'time', 'info': 'info'}
        

In [36]:
def row_parser(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    try:
        row['name'] = item0['name']
        row['address'] = item0['address']
        row['city'] = item0['city']
        row['state'] = item0['state']
        row['zip_code'] = '00000'
        logger.info("[+] Row Data Parsed")
    except Exception as e:
        logger.error("[-] Row Data Raised Exception: {}", e)
        print(e)
        row['name'] = 'name'
        row['address'] = 'address'
        row['city'] = 'city'
        row['state'] = 'state'

    # now add item1 to the row data
    row['day'] = '|'.join(item1['day'].replace(',', '-'))
    row['time'] = '|'.join(item1['time'].replace(',', '-'))
    row['info'] = '|'.join(item1['info'].replace(',', '-'))

    # now return the row data dictionary
    
    return row


In [37]:
new_csv_file = './meeting_details.csv'
headers = ["name", 'address', 'city', 'state', 'zip_code', 'day', 'time', 'info']
def csv_writer(row_data, csv_filename, headers=None):
    with open(csv_filename, 'a') as f:
        if headers is not None:
            writer = csv.DictWriter(f, fieldnames=headers, delimiter='|')
            # writer.writeheader(headers)
            for row in row_data:
                writer.writerow(row)
        else:
            # fieldnames = [k for k in row_data.keys()]
            # writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='|')
            # writer.writeheader(fieldnames)
            writer = csv.DictWriter(f, fieldnames=headers, delimiter='|')
            for row in row_data:
                writer.writerow(row)

#### BASIC COMPONENTS BUILT..

<hr>

## CREATE LINK LISTS:

In [38]:
# whole list of links
link_list = create_list_from_column_data(csv_filename, 'link')

# shortened list: first 1000
links_1000 = link_list[:1000]

# shortened list: first 500
links_500 = link_list[:500]

links_100 = link_list[:100]

####################################################################################
# chunk the links into bunches of 10,000
link_list1 = link_list[:10000]
link_list2 = link_list[10000:20000]
link_list3 = link_list[20000:30000]
link_list4 = link_list[30000:]


##### BUILD TWO FUNCTIONS PUTTING ALL THE ABOVE COMPONENTS TOGETHER
<hr>

In [47]:
def soup_list_scraper(soup_data):
    """
    :param soup_data: This is the list of soup data that was returned by the scrape function. 

    :returns: a list of dictionaries that will get stored to the variable row since each dict 
    creates a row of data for the csv file. 
    """
    try:
        rows = {'name':[], 'address':[], 'city':[], 'state':[], 'zip_code':[], 'day':[], 'time':[], 'info':[]}
        for soup in soup_data:
            # Create two dicts with the following keys
            address_dict = get_address_data(soup)
            details_dict = get_table_data(soup)
            logger.info("[+] Address and table data parsed")
            d = [address_dict, details_dict]
            row_data = parse_dicts(d[0], d[1])
            rows['name'].append(row_data['name'])
            rows['address'].append(row_data['address'])
            rows['city'].append(row_data['city'])
            rows['state'].append(row_data['state'])
            rows['zip_code'].append(row_data['zip_code'])
            rows['day'].append(row_data['day'])
            rows['time'].append(row_data['time'])
            rows['info'].append(row_data['info'])

        logger.info("++++++++ SCRAPED SCRAPED BATCH OF LINKS +++++++++ ")
        return rows

    except IndexError as ie:
        logger.error("{}: List Exhausted. No more Soup Items", ie)

    except Exception as e:
        logger.error("{}: Exception raised", e)
        

def single_soup_scraper(soup):
    """
    :param soup_data: This is the list of soup data that was returned by the scrape function. 

    :returns: a list of dictionaries that will get stored to the variable row since each dict 
    creates a row of data for the csv file. 
    """
    try:
        # Create two dicts with the following keys
        address_dict = get_address_data(soup)
        details_dict = get_table_data(soup)
        d = [address_dict, details_dict]
        row = row_parser(d[0], d[1])
        logger.info(
            "[+] Parsed Address and Table Data | Row [{}] Created", row)
        return row

    except IndexError as ie:
        logger.error("{}: List Exhausted. No more Soup Items", ie)

    except Exception as e:
        logger.error("{}: Exception raised", e)


In [40]:
def get_rows(link_list):
    """
    :param link_list: The list that is returned from the get_links function. Then it is passed to 
    scrape function that is used inside this function. 

    :returns: a list of dictionaries that will get stored to the variable row since each dict 
    creates a row of data for the csv file. 
    """
    try:
        soup_data = scrape(link_list)
        rows = soup_list_scraper(soup_data)
        return rows

    except requests.exceptions.RequestException as re:
        time.sleep(10)
        logger.error("{} Request Exception occured", re)
        
    except Exception as e:
        logger.error("{}: Exception while scraping link list", e)


In [40]:
def scrape(link_list, csv_file):
    """
    :param link_list: A list of links to scrape and get the soup data from the site, 
    parse the data and append the soup data to the soup_data list. 
    """
    # Two empty lists. One for soup_data and another to store row_data
    row_data = []
    soup_data = []
    # The headers for the csv_file
    headers = ["name", 'address', 'city', 'state',
               'zip_code', 'day', 'time', 'info']
    try:
        count = 0
        for link in link_list:
            soup = fetch_soup_data(link)
            soup_data.append(soup)
            logger.info(
                "[+] {} scraped successfully: Link number {}", link, count)
            count += 1
            if count % 500 == 0:
                time.sleep(5)
                # Every 500 links, write the row data to the new CSV file
                for soup in soup_data:
                    row = single_soup_scraper(soup)
                    row_data.append(row)
                # check that row_data has 500 items:
                    if len(row_data) == 500:
                        # after each soup item is parsed and the results appended to row_data, append row data to csv
                        csv_writer(row_data, csv_file, headers)
                        # Then clear both soup and row data lists
                        soup_data.clear()
                        row_data.clear()
                        # log the success of the script
                        logger.info(
                            "[+] Count is divisible by 500:{} | Writing ROW_DATA to [{}] | soup_data and row_data cleared!", count, csv_file)
        logger.info("<<<<<<<<<<<<<<< SUCCESS >>>>>>>>>>>>>>>>")
    except IndexError as e:
        logger.error("{}: List Exhausted. No more links to scrape.", e)


In [1]:
new_csv_filename = './meeting_details.csv'


In [48]:
def scrape(link_list):
    """
    :param link_list: A list of links to scrape and get the soup data from the site, 
    parse the data and append the soup data to the soup_data list. 
    """
    # Two empty lists. One for soup_data and another to store row_data
    dataframes = []
    soup_data = []
    # The headers for the csv_file
    headers = ["name", 'address', 'city', 'state',
               'zip_code', 'day', 'time', 'info']
    try:
        count = 0
        for link in link_list:
            soup = fetch_soup_data(link)
            soup_data.append(soup)
            logger.info(
                "[+] {} scraped successfully: Link number {}", link, count)
            count += 1
            if count % 10 == 0:
                time.sleep(5)
                # Every 10 links, write the row data to the new CSV file
                rows = soup_list_scraper(soup_data)
                row_df = pd.DataFrame.from_dict(rows)
                dataframes.append(row_df)
                soup_data.clear()
                logger.info(
                            "[+] Count is divisible by 10:{} | Writing ROW_DATA to [csv_file] | soup_data and row_data cleared!", count)
                # check that row_data has 500 items:
                    # if len(dataframes) > 0:
                    #     # after each soup item is parsed and the results appended to row_data, append row data to csv
                    #     csv_writer(row_data, csv_file, headers)
                    #     # Then clear both soup and row data lists
                    #     soup_data.clear()
                    #     row_data.clear()
                    #     # log the success of the script
        print(dataframes)
        logger.info("<<<<<<<<<<<<<<< SUCCESS >>>>>>>>>>>>>>>>")
    except IndexError as e:
        logger.error("{}: List Exhausted. No more links to scrape.", e)

In [49]:
scrape(links_100)

2022-06-15 06:34:25.384 | INFO     | __main__:fetch_soup_data:35 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/
2022-06-15 06:34:25.386 | INFO     | __main__:scrape:17 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-first-capital-group/ scraped successfully: Link number 0
2022-06-15 06:34:25.752 | INFO     | __main__:fetch_soup_data:35 - got soup data for link: https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/
2022-06-15 06:34:25.753 | INFO     | __main__:scrape:17 - [+] https://www.aa-meetings.com/aa-meeting/chillicothe-big-book-study/ scraped successfully: Link number 1
2022-06-15 06:34:26.107 | INFO     | __main__:fetch_soup_data:35 - got soup data for link: https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/
2022-06-15 06:34:26.109 | INFO     | __main__:scrape:17 - [+] https://www.aa-meetings.com/aa-meeting/waverly-pike-county-group/ scraped successfully: Link number 2
2022-06-15 06:34:26.406

[                                 name                     address  \
0  [ Chillicothe First Capital Group]     [ 165 West 4th Street ]   
1       [ Chillicothe Big Book Study]  [ 291 South Paint Street ]   
2        [ Waverly Pike County Group]   [ 104 South High Street ]   
3         [ One Step At A Time Group]       [ 211 Schmitt Drive ]   
4                  [ Searsport Group]      [ 7 Knox Bros Avenue ]   
5           [ East Millinocket Group]         [ 11 Maple Street ]   
6         [ Togus VA Speaker Meeting]             [ 1 VA Center ]   
7           [ Serenity Group Augusta]         [ 13 Green Street ]   
8               [ Looney Toons Group]        [ 393 Water Street ]   
9        [ 12th Step Being A Sponsor]    [ 6809 Guadalupe Street]   

                 city    state zip_code                  day  \
0       [Chillicothe]   [Ohio]  [00000]             [Friday]   
1       [Chillicothe]   [Ohio]  [00000]          [Wednesday]   
2           [Waverly]   [Ohio]  [00000]        

---

## Old Functions:


In [ ]:
# Scraped to 13180
new_csv_file = './meeting_details.csv'
headers = ["name", 'address', 'city', 'state',
           'zip_code', 'day', 'time', 'info']


def csv_writer(row_data, csv_filename, headers=None):
    with open(csv_filename, 'w') as f:
        if headers is not None:
            writer = csv.DictWriter(f, fieldnames=headers, delimiter='|')
            writer.writeheader(headers)
            for row in row_data:
                writer.writerow(row)
        else:
            fieldnames = [k for k in row_data.keys()]
            writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='|')
            writer.writeheader(fieldnames)
            for row in row_data:
                writer.writerow(row)


csv_writer(row_data1, csv_filename, headers)
